Tỉ lệ thất nghiệp qua từng năm

In [1]:
import pandas as pd

# Đường dẫn đến file Excel
file_path = "stat.xlsx"

# Đọc sheet cụ thể, ví dụ sheet tên là "SinhVien"
df = pd.read_excel(file_path, sheet_name="Inflation Rate")

print(df.head())  # Xem 5 dòng đầu tiên

        Date  Inflation Rate (%)  Unnamed: 2  \
0 2015-01-01                0.94         NaN   
1 2015-02-01                0.34         NaN   
2 2015-03-01                0.03         NaN   
3 2015-04-01                0.99         NaN   
4 2015-05-01                0.95         NaN   

                                          Unnamed: 3  
0  Nguồn: https://tradingeconomics.com/vietnam/in...  
1                                                NaN  
2                                                NaN  
3                                                NaN  
4                                                NaN  


In [10]:
inflation_by_months = []

In [13]:
for i in range(0, 10*12, 12):
  df_year = df.iloc[i:i+12, 1]
  inflation_by_months.append(df_year)

In [19]:
from functools import reduce

inflation_by_years = list(map(lambda x: reduce(lambda acc, e: acc * (1+e/100), x, 1), inflation_by_months))

In [20]:
inflation_by_years

[1.0690010402073449,
 1.369964024132872,
 1.5159911077514212,
 1.5178560095594011,
 1.3915344164927008,
 1.4645494460494344,
 1.243874618992762,
 1.4505587297123428,
 1.4674434974222954,
 1.5333212324409642]

πt​−πt−1​=α−bUt​+εt​

In [ ]:
import io
import pandas as pd
pd.read_csv(io.StringIO('''
"2,1425"
"2,09"
"2,0425"
2
"1,9875"
"2,245"
"2,90"
"2,13"
"2,07"
"2,05"
'''), header=None)


In [31]:
unemployment_rate_percent = [2.14, 2.09, 2.04, 2.00, 1.99, 2.25, 2.90, 2.13, 2.07, 2.05]
unemployment_rate = [x/100 for x in unemployment_rate_percent]

In [39]:
import pandas as pd
import statsmodels.api as sm

# Dữ liệu giả định
data = {
    "inflation_t": inflation_by_years[1:],
    "inflation_t_1": inflation_by_years[:-1],
    "unemployment": unemployment_rate[1:]
}
df = pd.DataFrame(data)

# Tạo biến thay đổi lạm phát
df["delta_inflation"] = df["inflation_t"] - df["inflation_t_1"]

# Hồi quy (π_t - π_{t-1}) = α - b * U_t + ε_t
X = sm.add_constant(df["unemployment"])
y = df["delta_inflation"]

model = sm.OLS(y, X).fit()
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:        delta_inflation   R-squared:                       0.280
Model:                            OLS   Adj. R-squared:                  0.177
Method:                 Least Squares   F-statistic:                     2.721
Date:                Sat, 11 Oct 2025   Prob (F-statistic):              0.143
Time:                        04:10:38   Log-Likelihood:                 5.7199
No. Observations:                   9   AIC:                            -7.440
Df Residuals:                       7   BIC:                            -7.045
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const            0.6965      0.394      1.768   

In [38]:
alpha = model.params['const']         # hệ số chặn α
beta = model.params['unemployment']   # hệ số dốc β

print("alpha =", alpha)
print("beta  =", beta)

nairu = alpha / beta
print(nairu)

alpha = 0.5172496942182111
beta  = -21.469913195339558
-0.024091839101170177


Okun Law

In [ ]:
import io
import pandas as pd
pd.read_csv(io.StringIO('''
-0.2,0.6,0.8,-0.3,0.4,-0.4,-1.9,0.0,-0.6,-0.3,-0.2
'''), header=None)


In [41]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

unemployment_t = np.array(unemployment_rate[1:])
unemployment_t_1 = np.array(unemployment_rate[:-1])
gap = np.array([-0.2,0.6,0.8,-0.3,0.4,-0.4,-1.9,0.0,-0.6,-0.3])

# 1. Tính Δu_t
du = unemployment_t - unemployment_t_1   # Δu_t = u_t - u_{t-1}

# 2. Cắt gap tương ứng (bỏ năm đầu tiên)
gap = gap[1:]

# 3. Tạo DataFrame cho rõ ràng
df = pd.DataFrame({
    "du": du,
    "gap": gap
})

# 4. Hồi quy OLS: Δu_t = α + β * gap_t
X = sm.add_constant(df["gap"])   # thêm cột constant (α)
y = df["du"]
model = sm.OLS(y, X).fit()

# 5. In kết quả
print(model.summary())

# 6. (Tùy chọn) In riêng hệ số
print(f"\nα = {model.params['const']:.4f}")
print(f"β = {model.params['gap']:.4f}")


                            OLS Regression Results                            
Dep. Variable:                     du   R-squared:                       0.336
Model:                            OLS   Adj. R-squared:                  0.242
Method:                 Least Squares   F-statistic:                     3.550
Date:                Sat, 11 Oct 2025   Prob (F-statistic):              0.102
Time:                        04:12:20   Log-Likelihood:                 40.010
No. Observations:                   9   AIC:                            -76.02
Df Residuals:                       7   BIC:                            -75.63
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0006      0.001     -0.547      0.6